In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **About Dataset**
##### The "Bitcoin Price| Daily Price | Weekly Update" dataset provides weekly updates on Bitcoin prices along with related information to aid in cryptocurrency market analysis and trend monitoring. This dataset includes historical data of Bitcoin prices over several weeks, encompassing opening price, closing price, highest price, lowest price, and trading volume.

# **Key Features:**

##### **Date:** A specific date within each day of trading, denoting when the Bitcoin price was recorded.
##### **Price:** Bitcoin's price at a specific time during the daily trading period, denoting its value in the respective currency (USD).
##### **Open:** Bitcoin's opening price at the start of each day's trading period, illustrating the starting price for Bitcoin trading during that day.
##### **High:** The peak price of Bitcoin reached during each day's trading period, offering insight into Bitcoin's highest value attained within that day.
##### **Low:** The lowest price of Bitcoin observed during each day's trading period, reflecting Bitcoin's lowest value within that day.
##### **Vol. (Volume):** Bitcoin's trading volume within a specific unit (e.g., number of Bitcoins traded) during each day's trading period, providing insight into trading activity in the Bitcoin market.
###### **Change %:** The percentage change in Bitcoin's price from the opening to the closing price within each day of trading, indicating the relative price shift compared to the opening price.


#### The "Bitcoin Price| Daily Price | Weekly Update" dataset is designed to assist market analysts, traders, and researchers in understanding the weekly dynamics of Bitcoin prices and identifying cryptocurrency market trends that may affect investment decisions. With these features, the dataset facilitates informed trading strategies and investment decisions in the dynamic cryptocurrency market.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/bitcoin-price-weekly-update/Bitcoin Historical Data3.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe().T

## convert objective column in float

In [ ]:
# import pandas as pd
# Load the dataset
filename = "/kaggle/input/bitcoin-price-weekly-update/Bitcoin Historical Data3.csv"
df = pd.read_csv(filename)

# Custom function to convert values with scale abbreviations to numeric values
def convert_to_numeric(value):
    if isinstance(value, str):
        scale_mapping = {'K': 1e3, 'M': 1e6, 'B': 1e9}
        if value[-1] in scale_mapping:
            return float(value[:-1]) * scale_mapping[value[-1]]
        else:
            return float(value)
    else:
        return value


# Convert 'Vol.' column to numeric
df["Vol."] = df["Vol."].apply(convert_to_numeric)

# Convert 'Change %' column to numeric
df["Change %"] = df["Change %"].str.replace('%','').astype('float64')

# Convert 'Price', 'Open', 'High', and 'Low' columns to numeric after removing commas
numeric_columns = ['Price', 'Open', 'High', 'Low']
for col in numeric_columns:
    df[col] = df[col].str.replace(',', '').astype('float64')


In [ ]:
#converting date column to datetime format
df['Date']=pd.to_datetime(df['Date'])

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## remove missing values

In [ ]:
df.fillna(method='ffill',inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

# **correlation metrix**

In [ ]:
correlation_metrix = df.corr()

plt.figure(figsize=(10,5))
sns.heatmap(correlation_metrix, annot=True, cmap='coolwarm',fmt='.2f')
plt.title('correlation matrix')
plt.show()

# **check price over time through plotly**

In [ ]:
import plotly.graph_objects as go

# Assuming df contains your DataFrame with 'Date' and 'Price' columns

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines'))

fig.update_layout(title='Price Over Time',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()


# **daily percent changes in bitcoin**

In [ ]:
# Calculate the daily percent change
df['Daily Percent Change'] = df['Price'].pct_change()

# Create a figure
fig = go.Figure()

# Add trace for the daily percent change
fig.add_trace(go.Scatter(x=df['Date'], y=df['Daily Percent Change'], name='Daily Percent Change', line=dict(color='blue'), mode='lines'))

# Update layout
fig.update_layout(title='Daily Percent Change Over Time',
                  xaxis_title='Date',
                  yaxis_title='Daily Percent Change')

# Show the plot
fig.show()

In [ ]:
df['Daily Percent Change'] = df['Price'].pct_change()

# Calculate the average daily percent change
average_daily_change = df['Daily Percent Change'].mean()

# Create a figure
fig = go.Figure()

# Add trace for the daily percent change
fig.add_trace(go.Scatter(x=df['Date'], y=df['Daily Percent Change'], name='Daily Percent Change', line=dict(color='blue'), mode='lines'))

# Add trace for the average daily percent change
fig.add_hline(y=average_daily_change, line_dash="dash", line_color="red", annotation_text=f'Average: {average_daily_change:.2%}', annotation_position="bottom right")

# Update layout
fig.update_layout(title='Daily Percent Change Over Time',
                  xaxis_title='Date',
                  yaxis_title='Daily Percent Change')

# Show the plot
fig.show()

# **volumne analysis**

In [ ]:
# Create a figure
# volumne analysis
fig = go.Figure()

# Add trace for trading volume
fig.add_trace(go.Scatter(x=df['Date'], y=df['Vol.'], name='Volume', line=dict(color='green')))

# Update layout
fig.update_layout(title='Trading Volume Over Time',
                  xaxis_title='Date',
                  yaxis_title='Volume')

# Show the plot
fig.show()

# **weekly average price**

In [ ]:
# Calculate weekly average prices
weekly_avg_prices = df.resample('W-Mon', on='Date').mean()

# Create a figure
fig = go.Figure()

# Add trace for weekly average bitcoin price
fig.add_trace(go.Scatter(x=weekly_avg_prices.index, y=weekly_avg_prices['Price'], line=dict(color='green')))

# Update layout
fig.update_layout(title='Weekly Average Bitcoin Prices',
                  xaxis_title='Week',
                  yaxis_title='Price (USD)',
                  xaxis=dict(tickformat='%Y-%m-%d'),  # Format x-axis ticks as dates
                  yaxis=dict(gridcolor='lightgrey'),  # Add gridlines
                  plot_bgcolor='rgba(0,0,0,0)')      # Set plot background color to transparent

# Show the plot
fig.show()

In [ ]:
# Create a histogram trace
histogram = go.Histogram(x=df['Change %'].dropna(), nbinsx=20, marker=dict(color='blue', line=dict(color='black', width=1)))

# Create a figure
fig = go.Figure(histogram)

# Update layout
fig.update_layout(title='Distribution of Daily Percentage Change in Bitcoin Prices',
                  xaxis_title='Percentage Change',
                  yaxis_title='Frequency',
                  xaxis=dict(gridcolor='lightgrey'),  # Add gridlines
                  yaxis=dict(gridcolor='lightgrey'),  # Add gridlines
                  plot_bgcolor='rgba(0,0,0,0)',      # Set plot background color to transparent
                  bargap=0.05)                       # Set gap between bars

# Show the plot
fig.show()

# **Observation:**
## 1. Analyze the trading price of Bitcoin over a four-year period.?
##### March, 2024: 69.4371k
##### Dec, 2023: 42.2725k
##### Dec, 2022: 16.9028k
##### Dec, 2021: 47.5452k

## 2. Determine the percentage change in the price of Bitcoin?
##### Average: -0.09%

## 3. Calculate the trading volume of Bitcoin in the year 2024?
##### Feb, 2024: 37.43k
